# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
!pip install gmaps

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [63]:
weather_data_df = pd.read_csv('weather_data.csv', index_col=0)
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pacaembu,-21.5622,-51.2606,28.62,52,96,2.97,BR,1637429146
1,mar del plata,-38.0023,-57.5575,29.01,55,0,4.02,AR,1637428987
2,busselton,-33.6500,115.3333,15.10,66,44,5.24,AU,1637429146
3,hermanus,-34.4187,19.2345,21.61,71,52,8.86,ZA,1637429147
4,punta arenas,-53.1500,-70.9167,8.06,53,40,18.01,CL,1637429147


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
lower_than_eighty_df = weather_df[weather_df["Max Temp"] < 80]
higher_than_seventy_df = lower_than_eighty_df[lower_than_eighty_df["Max Temp"] > 20]
wind_speed_less_df = higher_than_seventy_df[higher_than_seventy_df["Wind Speed"] < 10]
no_clouds = wind_speed_less_df[wind_speed_less_df["Cloudiness"] == 0]
no_clouds.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
lat = no_clouds["Lat"].tolist()
lng = no_clouds["Lng"].tolist()
hotel = []
for i in range(0, len(lat)):
    lat_test = str(lat[i])
    lng_test = str(lng[i])
    target_search = "Places to Stay"
    target_radius = 5000
    #target_coordinates = lat_test  lng_test
    params = {
    "location": f'{lat[i]}, {lng[i]}',
    "radius": target_radius,
    "type": "hotel",
    "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    j = 0
    try:
        while "lodging" not in places_data["results"][j]["types"]:
            j += 1
        hotel.append(places_data["results"][j]["name"])
    except IndexError:
        hotel.append("No Nearby Hotels")
    
hotel_df = no_clouds
hotel_df["Hotel Name"] = hotel
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
locations = weather_df[["Lat", "Lng"]]
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
heat_layer = gmaps.heatmap_layer(locations, 
                                 1  <= 9223372036854775807)
fig
# Display figure


TypeError: object of type 'bool' has no len()